In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://mlfromscratch.com/content/images/2020/01/model_stacking_overview-4.png)mlfromscratch.com

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/5802.csv", low_memory=False)
print(df.shape)
df.head()

In [ ]:
#Code by Parul Pandey  https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python


from sklearn.impute import SimpleImputer
df_most_frequent = df.copy()
#setting strategy to 'mean' to impute by the mean
mean_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
df_most_frequent.iloc[:,:] = mean_imputer.fit_transform(df_most_frequent)

In [ ]:
df_most_frequent.isnull().sum()

In [ ]:
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
# Lgbm
import lightgbm as lgb
import catboost
from catboost import Pool
import xgboost as xgb

# Suppr warning
import warnings
warnings.filterwarnings("ignore")

#Label Encoding

In [ ]:
#fill in mean for floats
for c in df_most_frequent.columns:
    if df_most_frequent[c].dtype=='float16' or  df_most_frequent[c].dtype=='float32' or  df_most_frequent[c].dtype=='float64':
        df_most_frequent[c].fillna(df[c].mean())

#fill in -999 for categoricals
df_most_frequent = df_most_frequent.fillna(-999)
# Label Encoding
for f in df_most_frequent.columns:
    if df_most_frequent[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df_most_frequent[f].values))
        df_most_frequent[f] = lbl.transform(list(df_most_frequent[f].values))
        
print('Labelling done.')

In [ ]:
import seaborn as sns
sns.distplot(df_most_frequent['time'])
plt.title('Time Distribution');

In [ ]:
print('skew',df_most_frequent['time'].skew())
print('kurtosis',df_most_frequent['time'].kurtosis())

In [ ]:
ax = sns.heatmap(df.corr(), annot=True, fmt=".4f")

In [ ]:
y = df_most_frequent['time']
df_most_frequent = df_most_frequent.drop(['time'], axis=1)

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

def plot_predict(pred, true):
    indexs = []
    for i in range(len(pred)):
        indexs.append(i)
        

    fig = go.Figure()

    fig.add_trace(go.Line(
        x=indexs,
        y=pred,
        name="Predict"
    ))

    fig.add_trace(go.Line(
        x=indexs,
        y=true,
        name="Test"
    ))

    fig.show()

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(
    df_most_frequent, y, random_state=42
)
   

#Lasso

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

lasso_params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
lasso = Lasso(random_state=42)
clf_lasso = GridSearchCV(lasso, lasso_params, cv=5, scoring='neg_mean_squared_error', n_jobs= 4, verbose = 1)
clf_lasso.fit(df_most_frequent, y)
print(clf_lasso.best_estimator_)
print(clf_lasso.best_score_)

#I changed max_features to 2. In the original code was 4. Due to the number of columns/features float (not integer)

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

import lightgbm as lgb
lightgbm_params ={'learning_rate':[0.0001, 0.001, 0.003, 0.01, 0.1],
                  'n_estimators':[10,20, 50, 100],
                 'max_depth':[4, 6, 10, 15, 20, 50]}
gbm = lgb.LGBMRegressor(random_state = 42)
clf_gbm = GridSearchCV(gbm, lightgbm_params, cv=5,  scoring='neg_mean_squared_error',n_jobs= 4, verbose = 1)
clf_gbm.fit(df_most_frequent, y)
print(clf_gbm.best_estimator_)
print(clf_gbm.best_score_)
# (learning_rate=0.001, max_depth=6, n_estimators=50, random_state=42)

In [ ]:
scores = {}

gbm = lgb.LGBMRegressor(random_state = 42, learning_rate=0.001, max_depth=6, n_estimators=50)
model = gbm.fit(df_most_frequent, y)
pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))
score = 100* max(0, 1-mean_squared_error(y_test, pred))
print(score)
scores['LGBM'] = score

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

from sklearn.ensemble import AdaBoostRegressor
adam_boosting_params = {'learning_rate':[0.0001, 0.001, 0.003, 0.01, 0.1,1],
                        'n_estimators':[10,20, 50, 100]}
ada = AdaBoostRegressor(random_state=42)
clf_ada = GridSearchCV(ada, adam_boosting_params, cv=5,  scoring='neg_mean_squared_error',n_jobs= 4, verbose = 1)
clf_ada.fit(df_most_frequent, y)
print(clf_ada.best_estimator_)
print(clf_ada.best_score_)
# (learning_rate=0.0001, n_estimators=100, random_state=42)

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

ada = AdaBoostRegressor(random_state=42, learning_rate=0.0001, n_estimators=100)
model = ada.fit(X_train, y_train)
pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))
score = 100* max(0, 1-mean_squared_error(y_test, pred))
print(score)
scores['ADA'] = score

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

from sklearn.svm import LinearSVR

svr_params = {'C':[0.0001, 0.001,0.01, 0.1, 1 , 10, 100]}
svr = LinearSVR(random_state=42)
clf_svr = GridSearchCV(svr, svr_params, cv=5, scoring='neg_mean_squared_error', n_jobs=4, verbose=1)
clf_svr.fit(df_most_frequent, y)
print(clf_svr.best_estimator_)
print(clf_svr.best_score_)
# (C=0.001, random_state=42)

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

lvr = LinearSVR(C=0.001, random_state=42)
model = svr.fit(X_train, y_train)
pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))
score = 100* max(0, 1-mean_squared_error(y_test, pred))
print(score)
scores['SVR'] = score

In [ ]:
#Code by Josué Nascimento https://www.kaggle.com/josutk/eda-stacking-regularization/notebook

result = pd.DataFrame([])
result['model'] = list(scores.keys())
result['score'] = list(scores.values())
result = result.sort_values(['score'], ascending=False)
result.head()

It was suppose to have Random Forest and XGBoost in my Stacking attempt. However, I had some issues with the snippets.  Maybe I'll try in another Notebook 